In [1]:
import os
import csv
import numpy as np

In [2]:
main_dir ="Results_union"
old_drifts = ["sea_P(y)","sea_P(y_x)","sea_P(x_y)","sine_P(y)","sine_P(y_x)","sine_P(x_y)"]
new_drifts = ["jitter","disappearing-minority","appearing-minority","appearing-clusters",
             "shapeshift","minority-share","splitting-clusters","clusters-movement","borderline"]
type_drift = ["incremental","sudden","periodic"]
unbalance_lvls = [1,2,3,4]
stats_w = ["Recall_0","Recall_1", "F1_0", "F1_1", "Gmean"]
stats_tot =["time", "memory"]
algs = ["RebalanceStream","CSMOTE","improved_UOB","improved_OOB","WEOB1","WEOB2","ESOS_ELM","UOB","OOB"]

In [3]:
def import_csv(csvfilename):
    data = []
    with open(csvfilename, "r") as scraped:
        reader = csv.reader(scraped, delimiter=',')
        row_index = 0
        for row in reader:
            if row:  # avoid blank lines
                row_index += 1
                columns = [str(row_index)]+ [row[i] for i in range(len(row))]
                data.append(columns)
        #print(f"{row_index} valid rows")
        
    return data

# MEAN ALL ROWS

# OLD

In [4]:
final_dict = {}

for old in old_drifts:
    path = main_dir + "/" + old[old.index("P"):]   #old = sea_P(y)
    if old not in final_dict:
        final_dict[old] = {}
        
    for ds in os.listdir(path):
        #check if ds in old (directory giusta)
        if old[:3] == ds[:3]:
            path1 = path + "/" + ds
            
            unbalance = 0
            for u in unbalance_lvls:
                if str(u) in ds:
                    unbalance = u
            assert unbalance != 0
                        
            if unbalance not in final_dict[old]:
                final_dict[old][unbalance] = {}
                
            dtype = "sudden"
            if "G" in ds:
                dtype = "incremental"
                
            if dtype not in final_dict[old][unbalance]:
                final_dict[old][unbalance][dtype] = {}

            for alg in os.listdir(path1):
                if "DS_Store" in alg:
                    continue
                path2 = path1 + "/" + alg
                class0_first = []
                class1_first = []
                class0_second = []
                class1_second = []
                
                print(f"{old} {unbalance} {dtype} {alg}")
                #prendo i valori di tutti i 10 esperimenti
                for result in os.listdir(path2):
                    if "err" not in result or "DS_Store" in result:
                        continue
                    start = False
                    first_half = True
                    data = import_csv(path2+"/"+ result)
                    daily0 = []
                    daily1 = []
                    for data_row in data:
                        if data_row[1] == "learning evaluation instances":
                            start = True
                            continue
                        if not start:
                            continue
                        if float(data_row[1]) > 50000 :
                            first_half = False
                            class0_first.append(np.mean(daily0))
                            class1_first.append(np.mean(daily1))
                            daily0 = []
                            daily1 = []
                        
                        try:
                            daily0.append(float(data_row[9]))
                        except:
                            if data_row[9] == "?":
                                daily0.append(0)
                            else:
                                print("ZERO")
                                print(data_row)
                        try:
                            daily1.append(float(data_row[10]))
                        except:
                            if data_row[10] == "?":
                                daily1.append(0)
                            else:
                                print("ONE")
                                print(data_row)
                    if len(daily0)>0 and len(daily1)>0:
                        class0_second.append(np.mean(daily0))
                        class1_second.append(np.mean(daily1))
                            
                        
                    
                assert len(class0_first) > 0
                assert len(class1_first) > 0
                assert len(class0_second) > 0
                assert len(class1_second) > 0
                
                final_dict[old][unbalance][dtype][alg] = {}
                final_dict[old][unbalance][dtype][alg]["class0_first"] = np.mean(class0_first)
                final_dict[old][unbalance][dtype][alg]["class1_first"] = np.mean(class1_first)

                final_dict[old][unbalance][dtype][alg]["class0_second"] = np.mean(class0_second)
                final_dict[old][unbalance][dtype][alg]["class1_second"] = np.mean(class1_second)


                print(np.mean(class0_first))
                print(np.mean(class1_first))

                print(np.mean(class0_second))
                print(np.mean(class1_second))
                    
                    
                
        

sea_P(y) 1 sudden improved_OOB
99.61356125257187
97.8705825427544
99.80891012109764
98.2782782782783
sea_P(y) 1 sudden WEOB1
99.55975970248731
97.45712421903599
99.73198100512684
97.60936415038191
sea_P(y) 1 sudden OOB
99.59426953742357
97.78496998248478
99.83232469046693
98.48164906988437
sea_P(y) 1 sudden ESOS_ELM
96.47276222234773
83.7212105037349
98.38273855826358
87.11389183965744
sea_P(y) 1 sudden improved_UOB
99.36989589045807
95.93132169900083
99.50623613727304
95.69108063417954
sea_P(y) 1 sudden WEOB2
99.21095333463325
94.78871919363078
99.37616480855289
94.62137977484846
sea_P(y) 1 sudden UOB
99.59426953742357
97.78496998248478
99.83232469046693
98.48164906988437
sea_P(y) 1 sudden RebalanceStream
98.78173330052812
89.39742449957834
99.27584227491501
93.03017424564385
sea_P(y) 3 incremental improved_OOB
91.23156675807124
80.40790335053626
92.45833205439087
80.21898397874567
sea_P(y) 3 incremental WEOB1
91.10652121686577
80.22914342420837
92.29775453706553
79.95196156925542
sea

83.19382905953816
79.14111101365368
83.51959898101734
80.58755202787725
sea_P(y_x) 4 incremental ESOS_ELM
81.49464005137735
72.76324159880164
81.89688301732905
76.4334330925129
sea_P(y_x) 4 incremental improved_UOB
82.90066565031195
77.62916802588796
83.19713057123644
79.85634360264837
sea_P(y_x) 4 incremental WEOB2
81.01952179566986
66.40356391078964
81.83054432818344
74.0378906460841
sea_P(y_x) 4 incremental UOB
82.4685168102753
77.50283076045051
83.05517297253505
79.40828984936499
sea_P(y_x) 4 incremental RebalanceStream
82.46550455017939
75.77909786681028
82.69567292395801
78.08722804190168
sea_P(y_x) 1 incremental improved_OOB
88.74233151456045
45.207349720800806
86.61086140777702
43.520612280396584
sea_P(y_x) 1 incremental WEOB1
88.97339417551979
44.652030826303424
86.74450832155648
43.55903487760704
sea_P(y_x) 1 incremental OOB
87.95397148707023
45.037180645757765
86.03551772420391
43.622320479095535
sea_P(y_x) 1 incremental ESOS_ELM
87.05339787764507
40.6091879839191
86.0428899

94.48852000911094
77.96165176994363
95.40647629457577
80.0
sea_P(x_y) 2 incremental WEOB1
93.5617288866927
76.16594527138554
94.3420339783572
77.38855271193029
sea_P(x_y) 2 incremental OOB
93.66582063798697
76.30632702184474
94.36097018555843
77.52977014677376
sea_P(x_y) 2 incremental ESOS_ELM
92.82051290218921
71.74743802984568
93.4854485048138
75.26343476739925
sea_P(x_y) 2 incremental improved_UOB
91.44689275219642
71.91650103072416
92.94630473548645
74.23224071967738
sea_P(x_y) 2 incremental WEOB2
91.04722833165091
72.1116973460842
94.88938903222265
78.26289983656316
sea_P(x_y) 2 incremental UOB
89.74655287649725
69.3995462270592
92.07736389684814
72.62376237623764
sea_P(x_y) 2 incremental RebalanceStream
94.92032520966325
76.78822454278344
95.67398457306219
79.75258773037113
sea_P(x_y) 4 sudden improved_OOB
86.18510672049194
81.48000532804194
86.77339000470326
82.1731270396544
sea_P(x_y) 4 sudden WEOB1
86.0186219591551
81.49868698822371
86.63471964741686
82.15403297370513
sea_P(x_

93.8299744881748
89.81766319177368
97.93405790483148
96.79194440504895
sine_P(y_x) 3 sudden improved_OOB
98.07178558735615
95.61326303379423
99.10836371923428
97.89876476906554
sine_P(y_x) 3 sudden WEOB1
97.70433465427054
94.87099576448965
98.97401504016895
97.59700544767888
sine_P(y_x) 3 sudden OOB
98.30387442064871
96.16338636779456
99.17605563409442
98.07480564545726
sine_P(y_x) 3 sudden ESOS_ELM
91.2040629699381
81.14604500134283
96.56293283430797
92.11695579206878
sine_P(y_x) 3 sudden improved_UOB
96.8730019592394
93.23609647669556
98.56027505193038
96.67163437655242
sine_P(y_x) 3 sudden WEOB2
73.64267349649437
56.22677942632903
84.37823046518699
69.97077732320281
sine_P(y_x) 3 sudden UOB
96.15385595715993
91.75675526026174
98.07371148862653
95.6188605108055
sine_P(y_x) 3 sudden RebalanceStream
97.33830265974005
93.90622058769264
98.52939086943549
96.50585733094763
sine_P(y_x) 3 incremental improved_OOB
95.58624705765833
90.47950462392438
98.96463967880119
97.55021131764968
sine_P

99.65755658985192
99.48878112275021
99.75424263558168
99.62793503946905
sine_P(x_y) 4 sudden ESOS_ELM
97.1808432413409
93.97590233411717
98.45452575673951
97.70282535037518
sine_P(x_y) 4 sudden improved_UOB
99.40167366544628
99.11917342505635
99.59871455450657
99.39664021230251
sine_P(x_y) 4 sudden WEOB2
99.37027586559212
99.0719021704597
99.52535598301274
99.28669753472658
sine_P(x_y) 4 sudden UOB
99.65755658985192
99.48878112275021
99.75424263558168
99.62793503946905
sine_P(x_y) 4 sudden RebalanceStream
99.26462896825116
98.91437734334505
99.48966870023438
99.22947569209146
sine_P(x_y) 1 sudden improved_OOB
99.80873081832198
98.24015454035023
99.8824729466028
98.91924959216965
sine_P(x_y) 1 sudden WEOB1
99.70084740451772
97.31350306710478
99.83687148358172
98.51319915039952
sine_P(x_y) 1 sudden OOB
99.81188973884925
98.28060536422078
99.89354624085163
99.0224032586558
sine_P(x_y) 1 sudden ESOS_ELM
98.14617967477905
83.0060714093088
98.70447334748677
88.72206540050976
sine_P(x_y) 1 su

In [5]:
old = final_dict.copy()

# NEW

In [6]:
final_dict = old.copy()

In [7]:
for new in new_drifts:
    path = main_dir + "/" + new 
    
    if new not in final_dict:
        final_dict[new] = {}
        
    for unbalance in range(1,5):
        if unbalance not in final_dict[new]:
            final_dict[new][unbalance] = {}


        path1 = path + "/" + str(unbalance)

        for alg in os.listdir(path1):
            if "DS_Store" in alg:
                continue
            path2 = path1 + "/" + alg
     
            
            alg_dtypes = {}
            alg_dtypes["incremental"] = {}
            alg_dtypes["sudden"] = {}
            alg_dtypes["periodic"] = {}
            alg_dtypes["incremental"]["class0_first"] = []
            alg_dtypes["sudden"]["class0_first"] = []
            alg_dtypes["periodic"]["class0_first"] = []
            alg_dtypes["incremental"]["class0_second"] = []
            alg_dtypes["sudden"]["class0_second"] = []
            alg_dtypes["periodic"]["class0_second"] = []
            alg_dtypes["incremental"]["class1_first"] = []
            alg_dtypes["sudden"]["class1_first"] = []
            alg_dtypes["periodic"]["class1_first"] = []
            alg_dtypes["incremental"]["class1_second"] = []
            alg_dtypes["sudden"]["class1_second"] = []
            alg_dtypes["periodic"]["class1_second"] = []
            
            first_half = True

            print(f"{new} {unbalance} {alg}")
            #prendo i valori di tutti i 10 esperimenti dei 3 dtype
            
            for result in os.listdir(path2):
                if "err" not in result or "DS_Store" in result:
                    continue
                daily0 = []
                daily1 = []
                data = import_csv(path2+"/"+ result)
                start = False
                
                for data_row in data:
                    if data_row[1] == "learning evaluation instances":
                        start = True
                        continue
                    if not start:
                        continue
                    if float(data_row[1]) > 50000 :
                        first_half = False
                        
                        alg_dtypes[result[:result.index("_")]]["class0_first"].append(np.mean(daily0))
                        alg_dtypes[result[:result.index("_")]]["class1_first"].append(np.mean(daily1))
                        daily0 = []
                        daily1 = []

                    try:
                        daily0.append(float(data_row[9]))
                    except:
                        if data_row[9] == "?":
                            daily0.append(0.0)
                        else:
                            print("ZERO")
                            print(data_row)
                    try:
                        daily1.append(float(data_row[10]))
                    except:
                        if data_row[10] == "?":
                            daily1.append(0.0)
                        else:
                            print("ONE")
                            print(data_row)
                        
                    
                
                try:
                    if len(daily0)>0 and len(daily1)>0:
                        alg_dtypes[result[:result.index("_")]]["class0_second"].append(np.mean(daily0))
                        alg_dtypes[result[:result.index("_")]]["class1_second"].append(np.mean(daily1))
                except:
                    print(f"error with {result}")
            for k,i in alg_dtypes.items():      
                
                if k not in final_dict[new][unbalance]:
                    final_dict[new][unbalance][k] = {}
                assert alg not in final_dict[new][unbalance][k]
                final_dict[new][unbalance][k][alg] = {}
                final_dict[new][unbalance][k][alg]["class0_first"] = np.mean(i["class0_first"])
                final_dict[new][unbalance][k][alg]["class1_first"] = np.mean(i["class1_first"])
                final_dict[new][unbalance][k][alg]["class0_second"] = np.mean(i["class0_second"])
                final_dict[new][unbalance][k][alg]["class1_second"] = np.mean(i["class1_second"])
                print(k)
                print(np.mean(i["class0_first"]))
                print(np.mean(i["class1_first"]))
                print(np.mean(i["class0_second"]))
                print(np.mean(i["class1_second"]))

jitter 1 improved_OOB
incremental
93.17702193810275
59.474552467849364
93.39473342846023
60.55770720371804
sudden
98.69456468587278
91.3727962944129
99.82533236913834
98.44784972812656
periodic
93.17702193810275
59.474552467849364
93.39473342846023
60.55770720371804
jitter 1 WEOB1
incremental
90.6942609888119
54.83621061217089
91.272266328641
56.552640318830946
sudden
98.22160980871186
88.82115241341872
99.71935764082232
97.53086419753086
periodic
90.6942609888119
54.83621061217089
91.272266328641
56.552640318830946
jitter 1 CSMOTE
incremental
84.48891444961619
40.928572599493265
84.23066104078761
40.67724867724867
sudden
87.71455450768954
49.42082233625739
89.43067484662579
53.43783783783785
periodic
84.48891444961619
40.928572599493265
84.23066104078761
40.67724867724867
jitter 1 OOB
incremental
92.16406306905543
57.54380491700861
92.73368190693347
59.4930235955877
sudden
98.44773996079546
90.58063803220493
99.80527211830554
98.2726285657882
periodic
92.16406306905543
57.543804917008

incremental
87.6487733702227
84.33804749739978
91.16099962285872
88.76101057520005
sudden
87.74213288868303
85.98743070092529
92.97258683811056
91.05783433700717
periodic
87.6487733702227
84.33804749739978
91.16099962285872
88.76101057520005
jitter 4 UOB
incremental
90.42722333233196
87.64515108928993
93.06071871127632
90.6549853984147
sudden
96.97740952720713
96.02575740842204
99.43280977312392
99.16353197386658
periodic
90.42722333233196
87.64515108928993
93.06071871127632
90.6549853984147
jitter 4 RebalanceStream
incremental
87.60856363408928
84.17193617328489
91.07083504408862
88.24543793272076
sudden
97.07975165491925
96.12541660278103
99.48514984319709
99.23955314458397
periodic
87.60856363408928
84.17193617328489
91.07083504408862
88.24543793272076
disappearing-minority 1 improved_OOB
incremental
98.59204539832965
10.305165769537725
99.99888887654308
0.0
sudden
98.43399225950948
5.544383775526842
99.73530119515522
0.0
periodic
92.77685915351452
55.74260089564907
92.4812817424996

incremental
86.0842431980277
14.892452222243275
86.16384816787038
0.0
sudden
84.5043249181119
8.204812556339741
84.54255761326375
0.0
periodic
89.64064199594662
85.71272485294011
90.54454040465475
88.53321532182721
disappearing-minority 4 WEOB2
incremental
88.87617640310303
14.948387666114433
89.61660166310989
0.0
sudden
83.718143628175
8.079373820631274
83.80334878748795
0.0
periodic
89.16342244249671
85.2468823053669
90.09896957601923
88.11072528977355
disappearing-minority 4 UOB
incremental
97.87895569605266
15.934386652845179
100.0
0.0
sudden
97.80045488675641
8.307693744697545
99.71217393920551
0.0
periodic
85.98646887838258
82.24630077685728
88.4238464679461
86.17747440273035
disappearing-minority 4 RebalanceStream
incremental
97.65884643583449
13.49138539703325
100.0
0.0
sudden
98.51290507513657
8.205211554169702
99.97999599919986
0.0
periodic
85.94418311384965
80.56329887955732
87.71146902511259
85.11621050562626
appearing-minority 1 improved_OOB
incremental
93.81611109422327
5

incremental
88.4302733980052
75.34150425479547
88.69436870912855
86.62024794390936
sudden
84.67545502442408
73.28361524341064
86.47987965400526
84.64331482272533
periodic
82.9624857913812
6.964637228323301
80.86255510106825
0.0
appearing-minority 4 WEOB2
incremental
87.46809874352985
74.654299525033
88.16334948953283
86.17608093460196
sudden
83.3038039509496
72.12300769581145
84.8319536108038
83.28498759826797
periodic
82.42931977315706
6.8696493428416705
80.19169329073483
0.0
appearing-minority 4 UOB
incremental
89.19722745678071
76.50219522577714
88.53446675057258
86.6202778513471
sudden
85.17412946920454
73.91481428961693
86.83546211521798
85.10592949401084
periodic
83.20611928205913
7.009658468834091
80.77733179668562
0.0
appearing-minority 4 RebalanceStream
incremental
89.74188960680932
75.32435100682157
89.11126995472956
86.1640932485619
sudden
85.85163560778832
72.22840886168825
87.19122177978775
84.02436009797972
periodic
98.82177317555742
6.371799306700407
100.0
0.0
appearing-

incremental
90.75659989805254
88.31389027789075
89.61961596232698
87.44537732606669
sudden
90.27193629580981
87.7704888676154
90.03602751191819
87.84245815017096
periodic
96.27302414108313
94.8137309724394
96.42651518059338
95.08629334459282
appearing-clusters 4 CSMOTE
incremental
90.55910902194873
86.96748206588602
89.27197047043116
86.01412921060715
sudden
90.36178651767433
86.5697730355913
89.95492256152104
86.677367576244
periodic
96.47901149367546
95.05323905548212
96.66131252868288
95.36919159770845
appearing-clusters 4 OOB
incremental
91.19476330230225
88.83366183137007
90.10003031221584
87.9081821547575
sudden
90.70883122819677
88.35189942240056
90.35754555705088
88.22144221797662
periodic
96.51510511086572
95.08181805623055
96.69885127657139
95.39820184071928
appearing-clusters 4 ESOS_ELM
incremental
80.3366972589011
74.85959177281823
80.07129971676953
77.2604234286086
sudden
79.73918314091749
74.58209767198593
80.26563881347235
77.75753157763008
periodic
85.84954444209
81.638

incremental
87.51915369247054
78.42127150936388
89.54668706111383
81.47714672908708
sudden
88.30378114645029
79.37834626712089
90.16673925216199
82.29528396997873
periodic
87.58005238093001
78.39287528781327
89.38735212097069
81.01155979793597
shapeshift 3 UOB
incremental
87.26686676018251
78.46193246756474
89.44242593627725
81.54616511968734
sudden
87.23410975248383
78.2929382302121
89.52470588235293
81.57793103448276
periodic
86.10350350783932
77.29986113586922
87.83843685171723
79.58691705604
shapeshift 3 RebalanceStream
incremental
89.9896556997098
77.62897749052608
90.94184084123069
80.32844657457007
sudden
89.75122632103758
77.99441414757277
90.929203539823
80.90062111801242
periodic
90.5170942746352
79.45519243957537
91.29475153364277
81.10069762525544
shapeshift 4 improved_OOB
incremental
88.30641772280842
85.51796975514871
89.92870770157649
87.52642706131078
sudden
88.11331093393768
84.95725609972195
89.81365121539623
87.19129531807303
periodic
88.12848645460839
85.45815177620

incremental
83.01897267343196
67.32885053506196
83.11566773124335
72.3816273372216
sudden
83.07466337436847
70.07076000380354
82.9952312745076
72.53611533302671
periodic
95.9974290117627
9.800943406856177
99.72613389510025
0.0
minority-share 3 improved_UOB
incremental
89.7055661158379
79.91012352179345
90.43339463548354
82.96149805327236
sudden
89.78575229524203
82.10971005905114
90.54290171606866
83.1142061281337
periodic
89.65546719967264
13.93745900670744
90.2439024390244
0.0
minority-share 3 WEOB2
incremental
89.12926348695231
79.07912189645043
90.17125967139008
82.5981222102509
sudden
89.30321505776003
81.424959392574
90.29948730774039
82.7725960470797
periodic
88.99647431511528
13.642595710368658
89.64096527369041
0.0
minority-share 3 UOB
incremental
88.94347158352853
79.08696802485137
89.97540511782867
82.409476293961
sudden
88.89213202549226
81.13653219148428
89.96200822631793
82.3866453638918
periodic
88.59010574241456
13.543901490539625
89.4280975489895
0.0
minority-share 3 R

incremental
90.90617144438764
82.95735259084367
90.55517241379312
82.88125000000001
sudden
89.43130403771589
80.93418640549989
89.80286794339091
81.85759546439886
periodic
94.35917421688886
88.62790600868719
95.09812053149311
90.05619678971632
splitting-clusters 3 CSMOTE
incremental
89.2664840983231
79.88897074846747
89.03067399799428
80.27731144686936
sudden
89.0182465512796
79.37956015396016
89.34842705131969
80.43348747482197
periodic
93.58193000961026
87.03795737937911
94.32745524955541
88.55964871115876
splitting-clusters 3 OOB
incremental
90.09103202560746
82.12842446292774
89.98538011695906
82.31932153392331
sudden
89.68433916713386
81.48094864640967
89.90444016953661
82.09977538060505
periodic
94.27786690178222
88.44038916412451
95.08289218208071
90.03732221512391
splitting-clusters 3 ESOS_ELM
incremental
82.35647740041445
68.86006243623659
81.6668115815487
70.42418612300719
sudden
81.7117226403679
67.97782095299253
81.80090972064961
70.85847997035269
periodic
86.34746138325686

incremental
83.51276032117886
56.590523347549194
87.9983068527113
66.71418389166071
sudden
81.17178213109624
47.711705402348755
86.62745683976041
62.67864589256605
periodic
85.45242861307278
64.56213570175875
88.15098535862113
69.61824805955963
clusters-movement 2 UOB
incremental
86.1794789480767
64.30882771485682
88.5280026938913
69.6152754692504
sudden
76.79393201699374
54.25409936402298
84.35597725129318
63.5285495369445
periodic
81.69076577394733
60.133595890641665
84.58813391631826
64.66671544596474
clusters-movement 2 RebalanceStream
incremental
92.3594660016738
62.92045512946812
93.05290511186
68.14373487246324
sudden
91.8892528607161
58.794267528621575
93.23492631733811
70.06618205489858
periodic
90.29325940906976
51.60121585901746
91.77491540224867
63.93011010052656
clusters-movement 3 improved_OOB
incremental
89.46747816643024
80.85070815798694
90.55038132456488
82.80011280669318
sudden
88.53445864448014
78.15307356892892
90.1478666276869
81.71041732126024
periodic
88.6763730

incremental
91.1558484871492
75.46759659430948
92.37804878048782
77.91060291060289
sudden
89.7328662397685
72.84876376806666
91.3362980638863
75.65157750342937
periodic
94.18312837756204
82.56614024194798
96.6855829322606
88.68840402059327
borderline 2 CSMOTE
incremental
89.31397246938235
71.63689195685774
89.47876004592422
72.14589665653493
sudden
87.04741527016387
68.00067992842932
86.8266081994631
67.51013899961869
periodic
91.70756686110592
76.36817807491111
92.80592663116241
78.76898761963298
borderline 2 OOB
incremental
91.32490715884714
75.8210773180344
92.45491460647318
78.04136253041362
sudden
90.04599725316501
73.44650129454232
91.54954345620561
76.04218600191754
periodic
94.65388462748359
83.91650106385045
97.30818113013552
90.56225904971711
borderline 2 ESOS_ELM
incremental
83.20396730080319
59.672536847211475
82.6772224126169
61.48131930538388
sudden
83.09040242957872
59.73901130923247
82.68746283538201
61.60993356602317
periodic
85.23490236187953
62.674822325270334
84.694

In [8]:
final_dict

{'sea_P(y)': {1: {'sudden': {'improved_OOB': {'class0_first': 99.61356125257187,
     'class1_first': 97.8705825427544,
     'class0_second': 99.80891012109764,
     'class1_second': 98.2782782782783},
    'WEOB1': {'class0_first': 99.55975970248731,
     'class1_first': 97.45712421903599,
     'class0_second': 99.73198100512684,
     'class1_second': 97.60936415038191},
    'OOB': {'class0_first': 99.59426953742357,
     'class1_first': 97.78496998248478,
     'class0_second': 99.83232469046693,
     'class1_second': 98.48164906988437},
    'ESOS_ELM': {'class0_first': 96.47276222234773,
     'class1_first': 83.7212105037349,
     'class0_second': 98.38273855826358,
     'class1_second': 87.11389183965744},
    'improved_UOB': {'class0_first': 99.36989589045807,
     'class1_first': 95.93132169900083,
     'class0_second': 99.50623613727304,
     'class1_second': 95.69108063417954},
    'WEOB2': {'class0_first': 99.21095333463325,
     'class1_first': 94.78871919363078,
     'class0_s

In [9]:
def mean_of(_dict ):
    values_0_first = []
    values_1_first = []
    values_0_second = []
    values_1_second = []
    for alg, values in _dict.items():
        values_0_first.append(values["class0_first"])
        values_1_first.append(values["class1_first"])
        values_0_second.append(values["class0_second"])
        values_1_second.append(values["class1_second"])
    return np.mean(values_0_first),np.mean(values_1_first),np.mean(values_0_second),np.mean(values_1_second)

In [10]:

with open('MEAN_F1XF1_HALF.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["drift","unbalance","type","alg","F1_0_first", "F1_1_first","F1_0_second", "F1_1_second"])
    for drift, dicts in final_dict.items():
        for unbalance, dtypes in dicts.items():
            for dtype, algs in dtypes.items():
                m0f,m1f,m0s,m1s = mean_of(algs)
                for alg, values in algs.items():
                    print([drift,unbalance,dtype,alg])
                    print(values)
                    writer.writerow([drift,
                                     unbalance,
                                     dtype,
                                     alg,
                                     values["class0_first"] - m0f,
                                     values["class1_first"] - m1f,
                                     values["class0_second"] - m0s,
                                     values["class1_second"] - m1s])

['sea_P(y)', 1, 'sudden', 'improved_OOB']
{'class0_first': 99.61356125257187, 'class1_first': 97.8705825427544, 'class0_second': 99.80891012109764, 'class1_second': 98.2782782782783}
['sea_P(y)', 1, 'sudden', 'WEOB1']
{'class0_first': 99.55975970248731, 'class1_first': 97.45712421903599, 'class0_second': 99.73198100512684, 'class1_second': 97.60936415038191}
['sea_P(y)', 1, 'sudden', 'OOB']
{'class0_first': 99.59426953742357, 'class1_first': 97.78496998248478, 'class0_second': 99.83232469046693, 'class1_second': 98.48164906988437}
['sea_P(y)', 1, 'sudden', 'ESOS_ELM']
{'class0_first': 96.47276222234773, 'class1_first': 83.7212105037349, 'class0_second': 98.38273855826358, 'class1_second': 87.11389183965744}
['sea_P(y)', 1, 'sudden', 'improved_UOB']
{'class0_first': 99.36989589045807, 'class1_first': 95.93132169900083, 'class0_second': 99.50623613727304, 'class1_second': 95.69108063417954}
['sea_P(y)', 1, 'sudden', 'WEOB2']
{'class0_first': 99.21095333463325, 'class1_first': 94.78871919

{'class0_first': 83.33022286062929, 'class1_first': 53.094430493729924, 'class0_second': 83.6459658331589, 'class1_second': 63.58584591116052}
['appearing-minority', 2, 'incremental', 'UOB']
{'class0_first': 90.72950552295379, 'class1_first': 63.19164798367436, 'class0_second': 89.11885371379586, 'class1_second': 71.19468309288376}
['appearing-minority', 2, 'incremental', 'RebalanceStream']
{'class0_first': 91.09889206422297, 'class1_first': 38.05520505971859, 'class0_second': 91.21221223954777, 'class1_second': 61.603627252117896}
['appearing-minority', 2, 'sudden', 'improved_OOB']
{'class0_first': 90.9290180708853, 'class1_first': 64.46944026624338, 'class0_second': 90.65654854664949, 'class1_second': 73.68822743751193}
['appearing-minority', 2, 'sudden', 'WEOB1']
{'class0_first': 89.71096918616882, 'class1_first': 62.80609238972581, 'class0_second': 89.48606163959678, 'class1_second': 71.98299556565397}
['appearing-minority', 2, 'sudden', 'CSMOTE']
{'class0_first': 87.8398287833511,

{'class0_first': 82.19308512231204, 'class1_first': 69.97467202838176, 'class0_second': 82.23689462437135, 'class1_second': 72.65139489710499}
['shapeshift', 3, 'sudden', 'improved_UOB']
{'class0_first': 88.01497824355265, 'class1_first': 78.694585940784, 'class0_second': 89.93720467545386, 'class1_second': 81.84724091520863}
['shapeshift', 3, 'sudden', 'WEOB2']
{'class0_first': 88.30378114645029, 'class1_first': 79.37834626712089, 'class0_second': 90.16673925216199, 'class1_second': 82.29528396997873}
['shapeshift', 3, 'sudden', 'UOB']
{'class0_first': 87.23410975248383, 'class1_first': 78.2929382302121, 'class0_second': 89.52470588235293, 'class1_second': 81.57793103448276}
['shapeshift', 3, 'sudden', 'RebalanceStream']
{'class0_first': 89.75122632103758, 'class1_first': 77.99441414757277, 'class0_second': 90.929203539823, 'class1_second': 80.90062111801242}
['shapeshift', 3, 'periodic', 'improved_OOB']
{'class0_first': 88.61029879960861, 'class1_first': 79.88618611398834, 'class0_se

['splitting-clusters', 4, 'periodic', 'CSMOTE']
{'class0_first': 93.85667668250022, 'class1_first': 91.61967455026955, 'class0_second': 94.72230327789126, 'class1_second': 92.97059204560175}
['splitting-clusters', 4, 'periodic', 'OOB']
{'class0_first': 93.70828245907276, 'class1_first': 91.7316131825372, 'class0_second': 94.82788849278242, 'class1_second': 93.13367988206997}
['splitting-clusters', 4, 'periodic', 'ESOS_ELM']
{'class0_first': 86.51865394427226, 'class1_first': 82.08970077457553, 'class0_second': 87.43702614861192, 'class1_second': 85.607037360252}
['splitting-clusters', 4, 'periodic', 'improved_UOB']
{'class0_first': 93.99936081890024, 'class1_first': 91.95284738395678, 'class0_second': 94.9216471594665, 'class1_second': 93.22719493854693}
['splitting-clusters', 4, 'periodic', 'WEOB2']
{'class0_first': 93.91939433235599, 'class1_first': 91.77742567144217, 'class0_second': 94.8141224124165, 'class1_second': 93.10978291805739}
['splitting-clusters', 4, 'periodic', 'UOB']
{